# 0. Import Python Librarys

In [ ]:
# importing utilities
import os
import sys
from datetime import datetime

# importing data science libraries
import pandas as pd
import random as rd
import numpy as np

# importing pytorch libraries
import torch
from torch import nn
from torch import autograd
from torch.utils.data import DataLoader

# import visualization libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from IPython.display import Image, display
sns.set_style('darkgrid')

# ignore potential warnings
import warnings
warnings.filterwarnings("ignore")

# 1. Prepare dataset from Raw Data File

In [ ]:
# file_path = r"C:/Users/vivek/Desktop/Anomaly_Detection/Dec-20-21 JE Data Dump.xlsx"
file_name = "Dec-20-21 JE Data Dump.xlsx"
df = pd.read_excel(file_name)
df.fillna(0, inplace=True)
df.head()

,JE Batch ID,JE Header ID,Journal Entry Name,Period,Source,Category,GL Date,LINE,Fund,Budget FY,...,Admin Control,DEBIT,CREDIT,Net Amount,IAC,USER_NAME,CREATION_DATE,LAST_UPDATE_DATE,Txn Num,Invoice_Num
0,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,1,050843XXXX0000,2021,...,52103,16.4,0.0,16.4,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
1,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,2,050843XXXX0000,2021,...,52103,6.3,0.0,6.3,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
2,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,3,050843XXXX0000,2021,...,52103,98.8,0.0,98.8,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
3,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,4,050843XXXX0000,2021,...,52103,6.2,0.0,6.2,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
4,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,5,050843XXXX0000,2021,...,52103,530.2,0.0,530.2,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0


In [ ]:
# seperate debit and credit card entries
debit_entries = df[df['DEBIT'] != 0]
credit_entries = df[df['CREDIT'] != 0]

In [ ]:
debit_entries

,JE Batch ID,JE Header ID,Journal Entry Name,Period,Source,Category,GL Date,LINE,Fund,Budget FY,...,Admin Control,DEBIT,CREDIT,Net Amount,IAC,USER_NAME,CREATION_DATE,LAST_UPDATE_DATE,Txn Num,Invoice_Num
0,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,1,050843XXXX0000,2021,...,52103,16.4,0.0,1.640000e+01,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
1,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,2,050843XXXX0000,2021,...,52103,6.3,0.0,6.300000e+00,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
2,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,3,050843XXXX0000,2021,...,52103,98.8,0.0,9.880000e+01,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
3,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,4,050843XXXX0000,2021,...,52103,6.2,0.0,6.200000e+00,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
4,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,5,050843XXXX0000,2021,...,52103,530.2,0.0,5.302000e+02,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200900,49631541,49675862,"Reverses ""RITM0014485 - Rvrs Risk Corridor Cle...",Dec-20-21,Manual ADI,Adjustment,2020-12-31 00:00:00,4,05015020156RB0,2015,...,162203,50,0.0,5.000000e+01,150.0,CKTB,2021-01-12 14:05:55,2021-01-12 15:06:27,0,0
200902,49631551,49675874,Dec-20-21 Budget Distribution USD,Dec-20-21,Budget Execution,Budget Distribution,2020-12-31 00:00:00,2,05020120211DAD,2021,...,0,2678868563,0.0,2.678869e+09,201.0,M179,2021-01-12 15:37:10,2021-01-12 15:37:26,0,0
200904,49631551,49675874,Dec-20-21 Budget Distribution USD,Dec-20-21,Budget Execution,Budget Distribution,2020-12-31 00:00:00,4,05020820211DB0,2021,...,0,254213096324,0.0,2.542131e+11,208.0,M179,2021-01-12 15:37:10,2021-01-12 15:37:26,0,0
200906,49631551,49675874,Dec-20-21 Budget Distribution USD,Dec-20-21,Budget Execution,Budget Distribution,2020-12-31 00:00:00,6,05020820211DB0,2021,...,0,254213096324,0.0,2.542131e+11,208.0,M179,2021-01-12 15:37:10,2021-01-12 15:37:26,0,0


In [ ]:
credit_entries

,JE Batch ID,JE Header ID,Journal Entry Name,Period,Source,Category,GL Date,LINE,Fund,Budget FY,...,Admin Control,DEBIT,CREDIT,Net Amount,IAC,USER_NAME,CREATION_DATE,LAST_UPDATE_DATE,Txn Num,Invoice_Num
13,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,14,050843XXXX0000,2021,...,52103,0,3.467900e+03,-3.467900e+03,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
14,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,15,050843XXXX0000,2021,...,52103,0,1.890642e+05,-1.890642e+05,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
15,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,16,050843XXXX0000,2021,...,52103,0,1.860021e+05,-1.860021e+05,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
16,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,17,050843XXXX0000,2021,...,52103,0,6.030000e+01,-6.030000e+01,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
17,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,18,050843XXXX0000,2021,...,52103,0,2.178800e+03,-2.178800e+03,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200898,49631541,49675862,"Reverses ""RITM0014485 - Rvrs Risk Corridor Cle...",Dec-20-21,Manual ADI,Adjustment,2020-12-31 00:00:00,2,05015020156RB0,2015,...,162203,0,5.000000e+01,-5.000000e+01,150.0,CKTB,2021-01-12 14:05:55,2021-01-12 15:06:27,0,0
200901,49631551,49675874,Dec-20-21 Budget Distribution USD,Dec-20-21,Budget Execution,Budget Distribution,2020-12-31 00:00:00,1,05020120211DAD,2021,...,0,0,2.678869e+09,-2.678869e+09,201.0,M179,2021-01-12 15:37:10,2021-01-12 15:37:26,0,0
200903,49631551,49675874,Dec-20-21 Budget Distribution USD,Dec-20-21,Budget Execution,Budget Distribution,2020-12-31 00:00:00,3,05020820211DB0,2021,...,0,0,2.542131e+11,-2.542131e+11,208.0,M179,2021-01-12 15:37:10,2021-01-12 15:37:26,0,0
200905,49631551,49675874,Dec-20-21 Budget Distribution USD,Dec-20-21,Budget Execution,Budget Distribution,2020-12-31 00:00:00,5,05020820211DB0,2021,...,0,0,2.542131e+11,-2.542131e+11,208.0,M179,2021-01-12 15:37:10,2021-01-12 15:37:26,0,0


In [ ]:
col_types = {'JE Batch ID' : 1,
 'JE Header ID' : 1,
 'Journal Entry Name' : 1,
 'Period' : 1,
 'Source' : 2,
 'Category' : 2,
 'GL Date' : 1,
 'LINE' : 1,
 'Fund' : 2,
 'Budget FY' : 1,
 ' Budget Activity Program' : 2,
 'Organization' : 2,
 'Object Class' : 2,
 'SGL Account' : 2,
 'Admin Control ' : 2,
 'DEBIT' : 1,
 'CREDIT' : 1,
 'Net Amount' : 1,
 'IAC' : 2,
 'USER_NAME' : 2,
 'CREATION_DATE' : 1,
 'LAST_UPDATE_DATE' : 1,
 'Txn Num' : 2,
 'Invoice_Num' : 1}

In [ ]:
print(col_types) # made a dictionary with some business logic which i do not know as of now

{'JE Batch ID': 1, 'JE Header ID': 1, 'Journal Entry Name': 1, 'Period': 1, 'Source': 2, 'Category': 2, 'GL Date': 1, 'LINE': 1, 'Fund': 2, 'Budget FY': 1, ' Budget Activity Program': 2, 'Organization': 2, 'Object Class': 2, 'SGL Account': 2, 'Admin Control ': 2, 'DEBIT': 1, 'CREDIT': 1, 'Net Amount': 1, 'IAC': 2, 'USER_NAME': 2, 'CREATION_DATE': 1, 'LAST_UPDATE_DATE': 1, 'Txn Num': 2, 'Invoice_Num': 1}


In [ ]:
#save debit and credit entries to seperate files
debit_entries.to_excel(r"debit_data.xlsx", index=False)
credit_entries.to_excel(r"credit_data.xlsx", index=False)

In [ ]:
#load debit card entries from file
debit_file_name = r"debit_data.xlsx"
df = pd.read_excel(debit_file_name)
df.head()

,JE Batch ID,JE Header ID,Journal Entry Name,Period,Source,Category,GL Date,LINE,Fund,Budget FY,...,Admin Control,DEBIT,CREDIT,Net Amount,IAC,USER_NAME,CREATION_DATE,LAST_UPDATE_DATE,Txn Num,Invoice_Num
0,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,1,050843XXXX0000,2021,...,52103,16.4,0.0,16.4,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
1,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,2,050843XXXX0000,2021,...,52103,6.3,0.0,6.3,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
2,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,3,050843XXXX0000,2021,...,52103,98.8,0.0,98.8,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
3,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,4,050843XXXX0000,2021,...,52103,6.2,0.0,6.2,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0
4,49593172,49624179,Dec-20-21 Misc Receipts USD,Dec-20-21,Receivables,Misc Receipts,2020-12-31 00:00:00,5,050843XXXX0000,2021,...,52103,530.2,0.0,530.2,843.0,AUTOSYSCMSHQ,2020-11-23 19:06:15,2020-11-23 19:06:33,0,0


In [ ]:
#select columns we want to train with
df_reduced = df[['JE Batch ID', 'Period', 'JE Header ID', 'Source', 'Category', 'LINE', 'Fund', 'Budget FY', ' Budget Activity Program', 'Organization', 'Object Class', ' SGL Account', 'Admin Control ', 'DEBIT', 'CREDIT', 'Net Amount', 'IAC', 'USER_NAME', 'CREATION_DATE', 'LAST_UPDATE_DATE']]

#save the reduced df to file
df_reduced.to_excel(r"data_reduced_new.xlsx", index=False)

# 2. Check if CUDA is available (needed to train on GPU), check for python and pytorch version

In [ ]:
# print CUDNN backend version
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] The CUDNN backend version: {}'.format(now, torch.backends.cudnn.version()))

[LOG 20240418-14:15:43] The CUDNN backend version: 8906


In [ ]:
#parameter to setup to use GPU
USE_CUDA = True

In [ ]:
# print current Python version
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] The Python version: {}'.format(now, sys.version))

[LOG 20240418-14:17:01] The Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
# print current PyTorch version
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] The PyTorch version: {}'.format(now, torch.__version__))

[LOG 20240418-14:18:38] The PyTorch version: 2.2.1+cu121


# 3. Initialize seed value (use same so we can reproduce results)

In [ ]:
# init deterministic seed
seed_value = 1234
rd.seed(seed_value) # set random seed
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    torch.cuda.manual_seed(seed_value) # set pytorch seed GPU

# 4. Load Dataset

In [ ]:
ori_dataset = pd.read_excel(r"C:\code_repo\neuron_ai\ai_ledger\data_reduced_new.xlsx", header=0)
ori_dataset.rename(columns=lambda x:x.replace(' ', '_'), inplace=True)

In [ ]:
#remove debit transaction that are negative numbers or not numbers
is_integer = pd.to_numeric(ori_dataset['DEBIT'], errors='coerce').notna()
#ori_dataset = ori_dataset[is_integer]

is_integer = pd.to_numeric(ori_dataset['CREDIT'], errors='coerce').notna()
#ori_dataset = ori_dataset[is_integer]

is_integer = pd.to_numeric(ori_dataset['Net_Amount'], errors='coerce').notna()
#ori_dataset = ori_dataset[is_integer]
#ori_dataset = ori_dataset[ori_dataset['DEBIT'] >= 0]


In [ ]:
# inspect the dataset dimensions
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] Transactional dataset of {} rows and {} columns loaded'.format(now, ori_dataset.shape[0], ori_dataset.shape[1]))

In [ ]:
# inspect top rows of the ERP dataset
ori_dataset.head(10)


In [ ]:
ori_dataset['CREATION_DATE'] = pd.to_datetime(ori_dataset['CREATION_DATE'])
ori_dataset['CREATION_DATE'] = (ori_dataset['CREATION_DATE'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
ori_dataset['LAST_UPDATE_DATE'] = pd.to_datetime(ori_dataset['LAST_UPDATE_DATE'])
ori_dataset['LAST_UPDATE_DATE'] = (ori_dataset['LAST_UPDATE_DATE'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [ ]:
# ncheck counts of categorical variables
ori_dataset.JE_Header_ID.value_counts()

# 5. Preprocess Categorical Variables

In [ ]:
# prepare to plot posting key and general ledger account side by side
fig, ax = plt.subplots(1,2)
fig.set_figwidth(12)

# plot the distribution of the posting key attribute
g = sns.countplot(x=ori_dataset.loc[:, 'Source'], ax=ax[0])
g.set_xticklabels(g.get_xticklabels(), rotation=0)
g.set_title('Distribution of Source attribute values')

# plot the distribution of the general ledger account attribute
g = sns.countplot(x=ori_dataset.loc[:, 'Category'], ax=ax[1])
g.set_xticklabels(g.get_xticklabels(), rotation=0)
g.set_title('Distribution of Category attribute values');

In [ ]:
# select categorical attributes to be "one-hot" encoded
categorical_attr_names = ['JE_Batch_ID', 'JE_Header_ID', 'Period',  'Source', 'Category', 'LINE', 'Fund', '_Budget_Activity_Program', 'Organization', 'Object_Class', '_SGL_Account', 'Admin_Control_', 'Admin_Control_', 'IAC', 'USER_NAME']

#convert any integer type categorical columns to category type to perform one hot encoding
ori_dataset['Budget_FY'] = ori_dataset['Budget_FY'].astype('category')
ori_dataset['Admin_Control_'] = ori_dataset['Admin_Control_'].astype('category')
ori_dataset['IAC'] = ori_dataset['IAC'].astype('category')
ori_dataset['USER_NAME'] = ori_dataset['USER_NAME'].astype('category')

# encode categorical attributes into a binary one-hot encoded representation
ori_dataset_categ_transformed = pd.get_dummies(ori_dataset[categorical_attr_names])
ori_dataset_categ_transformed = ori_dataset_categ_transformed.astype(int)

In [ ]:
ori_dataset.dtypes

In [ ]:
# inspect encoded sample transactions
ori_dataset_categ_transformed.head(10)

# 6. Preprocess Numerical Variables

In [ ]:
# init the plots
fig, ax = plt.subplots(1,1)
fig.set_figwidth(20)

# plot distribution of the local amount attribute
g = sns.distplot(ori_dataset['DEBIT'].tolist(), ax=ax)
g.set_title('Distribution of DEBIT amount values')

# set axis-labels
ax.set_xlabel('DEBIT')
ax.set_ylabel('density')



In [ ]:
def apply_log(x):
    if isinstance(x, str):
        return x
    else:
        return np.log(x)

In [ ]:
# select all the numerical attributes
numeric_attr_names = ['DEBIT', 'CREDIT']

# add a small epsilon to eliminate zero values from data for log scaling
numeric_attr = ori_dataset[numeric_attr_names] + 1e-4
numeric_attr = numeric_attr.map(apply_log)

# normalize all numeric attributes to the range [0,1]
ori_dataset_numeric_attr = (numeric_attr - numeric_attr.min()) / (numeric_attr.max() - numeric_attr.min())

In [ ]:
#inspect the normalized numerical attributes
ori_dataset_numeric_attr

In [ ]:
# init the plots
fig, ax = plt.subplots(1,1)
fig.set_figwidth(20)

# plot distribution of the local amount attribute
g = sns.distplot(ori_dataset_numeric_attr['DEBIT'].tolist(), ax=ax)
g.set_title('Distribution of scaled DEBIT amount values')

# set axis-labels
ax.set_xlabel('DEBIT')
ax.set_ylabel('density')


# 7. Merge Categorical and Numeric Variables

In [ ]:
# merge categorical and numeric subsets
ori_subset_transformed = pd.concat([ori_dataset_categ_transformed, ori_dataset_numeric_attr], axis = 1)

In [ ]:
# inspect final dimensions of pre-processed transactional data
ori_subset_transformed.shape

In [ ]:
ori_subset_transformed

# 8. Auto Encoder Neural Network Architecture

## 8.1. AutoEncoder Neural Networks - Encoder



In [ ]:
# implementation of the encoder network
class encoder(nn.Module):

    def __init__(self):

        super(encoder, self).__init__()

        # specify layer 1 - in 618, out 512
        self.encoder_L1 = nn.Linear(in_features=ori_subset_transformed.shape[1], out_features=512, bias=True) # add linearity
        nn.init.xavier_uniform_(self.encoder_L1.weight) # init weights according to [9]
        self.encoder_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify layer 2 - in 512, out 256
        self.encoder_L2 = nn.Linear(512, 256, bias=True)
        nn.init.xavier_uniform_(self.encoder_L2.weight)
        self.encoder_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 3 - in 256, out 128
        self.encoder_L3 = nn.Linear(256, 128, bias=True)
        nn.init.xavier_uniform_(self.encoder_L3.weight)
        self.encoder_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 4 - in 128, out 64
        self.encoder_L4 = nn.Linear(128, 64, bias=True)
        nn.init.xavier_uniform_(self.encoder_L4.weight)
        self.encoder_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 5 - in 64, out 32
        self.encoder_L5 = nn.Linear(64, 32, bias=True)
        nn.init.xavier_uniform_(self.encoder_L5.weight)
        self.encoder_R5 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 6 - in 32, out 16
        self.encoder_L6 = nn.Linear(32, 16, bias=True)
        nn.init.xavier_uniform_(self.encoder_L6.weight)
        self.encoder_R6 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 7 - in 16, out 8
        self.encoder_L7 = nn.Linear(16, 8, bias=True)
        nn.init.xavier_uniform_(self.encoder_L7.weight)
        self.encoder_R7 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 8 - in 8, out 4
        self.encoder_L8 = nn.Linear(8, 4, bias=True)
        nn.init.xavier_uniform_(self.encoder_L8.weight)
        self.encoder_R8 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 9 - in 4, out 3
        self.encoder_L9 = nn.Linear(4, 3, bias=True)
        nn.init.xavier_uniform_(self.encoder_L9.weight)
        self.encoder_R9 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # init dropout layer with probability p
        self.dropout = nn.Dropout(p=0.0, inplace=True)

    def forward(self, x):

        # define forward pass through the network
        x = self.encoder_R1(self.dropout(self.encoder_L1(x)))
        x = self.encoder_R2(self.dropout(self.encoder_L2(x)))
        x = self.encoder_R3(self.dropout(self.encoder_L3(x)))
        x = self.encoder_R4(self.dropout(self.encoder_L4(x)))
        x = self.encoder_R5(self.dropout(self.encoder_L5(x)))
        x = self.encoder_R6(self.dropout(self.encoder_L6(x)))
        x = self.encoder_R7(self.dropout(self.encoder_L7(x)))
        x = self.encoder_R8(self.dropout(self.encoder_L8(x)))
        x = self.encoder_R9(self.encoder_L9(x)) # don't apply dropout to the AE bottleneck

        return x

In [ ]:
# init training network classes / architectures
encoder_train = encoder()

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None and USE_CUDA == True):
    encoder_train = encoder().cuda()

In [ ]:
# print the initialized architectures
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] encoder architecture:\n\n{}\n'.format(now, encoder_train))

## 8.2. AutoEncoder Neural Network - Decoder

In [ ]:
# implementation of the decoder network
class decoder(nn.Module):

    def __init__(self):

        super(decoder, self).__init__()

        # specify layer 1 - in 3, out 4
        self.decoder_L1 = nn.Linear(in_features=3, out_features=4, bias=True) # add linearity
        nn.init.xavier_uniform_(self.decoder_L1.weight)  # init weights according to [9]
        self.decoder_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True) # add non-linearity according to [10]

        # specify layer 2 - in 4, out 8
        self.decoder_L2 = nn.Linear(4, 8, bias=True)
        nn.init.xavier_uniform_(self.decoder_L2.weight)
        self.decoder_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 3 - in 8, out 16
        self.decoder_L3 = nn.Linear(8, 16, bias=True)
        nn.init.xavier_uniform_(self.decoder_L3.weight)
        self.decoder_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 4 - in 16, out 32
        self.decoder_L4 = nn.Linear(16, 32, bias=True)
        nn.init.xavier_uniform_(self.decoder_L4.weight)
        self.decoder_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 5 - in 32, out 64
        self.decoder_L5 = nn.Linear(32, 64, bias=True)
        nn.init.xavier_uniform_(self.decoder_L5.weight)
        self.decoder_R5 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 6 - in 64, out 128
        self.decoder_L6 = nn.Linear(64, 128, bias=True)
        nn.init.xavier_uniform_(self.decoder_L6.weight)
        self.decoder_R6 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 7 - in 128, out 256
        self.decoder_L7 = nn.Linear(128, 256, bias=True)
        nn.init.xavier_uniform_(self.decoder_L7.weight)
        self.decoder_R7 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 8 - in 256, out 512
        self.decoder_L8 = nn.Linear(256, 512, bias=True)
        nn.init.xavier_uniform_(self.decoder_L8.weight)
        self.decoder_R8 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # specify layer 9 - in 512, out 618
        self.decoder_L9 = nn.Linear(in_features=512, out_features=ori_subset_transformed.shape[1], bias=True)
        nn.init.xavier_uniform_(self.decoder_L9.weight)
        self.decoder_R9 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        # init dropout layer with probability p
        self.dropout = nn.Dropout(p=0.0, inplace=True)

    def forward(self, x):

        # define forward pass through the network
        x = self.decoder_R1(self.dropout(self.decoder_L1(x)))
        x = self.decoder_R2(self.dropout(self.decoder_L2(x)))
        x = self.decoder_R3(self.dropout(self.decoder_L3(x)))
        x = self.decoder_R4(self.dropout(self.decoder_L4(x)))
        x = self.decoder_R5(self.dropout(self.decoder_L5(x)))
        x = self.decoder_R6(self.dropout(self.decoder_L6(x)))
        x = self.decoder_R7(self.dropout(self.decoder_L7(x)))
        x = self.decoder_R8(self.dropout(self.decoder_L8(x)))
        x = self.decoder_R9(self.decoder_L9(x)) # don't apply dropout to the AE output

        return x

In [ ]:
# init training network classes / architectures
decoder_train = decoder()

# push to cuda if cudnn is available
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    decoder_train = decoder().cuda()

# print the initialized architectures
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] decoder architecture:\n\n{}\n'.format(now, decoder_train))

# 9. Training

In [ ]:
# define the optimization criterion / loss function
loss_function = nn.BCEWithLogitsLoss(reduction='mean')

In [ ]:
# define learning rate and optimization strategy
learning_rate = 1e-3
encoder_optimizer = torch.optim.Adam(encoder_train.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.Adam(decoder_train.parameters(), lr=learning_rate)

In [ ]:
# specify training parameters
num_epochs = 50
mini_batch_size = 128

In [ ]:
# convert pre-processed data to pytorch tensor
torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

# convert to pytorch tensor - none cuda enabled
dataloader = DataLoader(torch_dataset, batch_size=mini_batch_size, shuffle=True, num_workers=0)
# note: we set num_workers to zero to retrieve deterministic results

# determine if CUDA is available at compute node
if (torch.backends.cudnn.version() != None) and (USE_CUDA == True):
    dataloader = DataLoader(torch_dataset.cuda(), batch_size=mini_batch_size, shuffle=True)

In [ ]:
# init collection of mini-batch losses
losses = []

# convert encoded transactional data to torch Variable
data = autograd.Variable(torch_dataset)

# train autoencoder model
for epoch in range(num_epochs):

    # init mini batch counter
    mini_batch_count = 0

    # determine if CUDA is available at compute node
    if(torch.backends.cudnn.version() != None) and (USE_CUDA == True):

        # set networks / models in GPU mode
        encoder_train.cuda()
        decoder_train.cuda()

    # set networks in training mode (apply dropout when needed)
    encoder_train.train()
    decoder_train.train()

    # start timer
    start_time = datetime.now()

    # iterate over all mini-batches
    for mini_batch_data in dataloader:

        # increase mini batch counter
        mini_batch_count += 1

        # convert mini batch to torch variable
        mini_batch_torch = autograd.Variable(mini_batch_data)

        # =================== (1) forward pass ===================================

        # run forward pass
        z_representation = encoder_train(mini_batch_torch) # encode mini-batch data
        mini_batch_reconstruction = decoder_train(z_representation) # decode mini-batch data

        # =================== (2) compute reconstruction loss ====================

        # determine reconstruction loss
        reconstruction_loss = loss_function(mini_batch_reconstruction, mini_batch_torch)

        # =================== (3) backward pass ==================================

        # reset graph gradients
        decoder_optimizer.zero_grad()
        encoder_optimizer.zero_grad()

        # run backward pass
        reconstruction_loss.backward()

        # =================== (4) update model parameters ========================

        # update network parameters
        decoder_optimizer.step()
        encoder_optimizer.step()

        # =================== monitor training progress ==========================

        # print training progress each 1'000 mini-batches
        if mini_batch_count % 1000 == 0:

            # print the training mode: either on GPU or CPU
            mode = 'GPU' if (torch.backends.cudnn.version() != None) and (USE_CUDA == True) else 'CPU'

            # print mini batch reconstuction results
            now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
            end_time = datetime.now() - start_time
            print('[LOG {}] training status, epoch: [{:04}/{:04}], batch: {:04}, loss: {}, mode: {}, time required: {}'.format(now, (epoch+1), num_epochs, mini_batch_count, np.round(reconstruction_loss.item(), 4), mode, end_time))

            # reset timer
            start_time = datetime.now()

    # =================== evaluate model performance =============================

    # set networks in evaluation mode (don't apply dropout)
    encoder_train.cpu().eval()
    decoder_train.cpu().eval()

    # reconstruct encoded transactional data
    reconstruction = decoder_train(encoder_train(data))

    # determine reconstruction loss - all transactions
    reconstruction_loss_all = loss_function(reconstruction, data)

    # collect reconstruction loss
    losses.extend([reconstruction_loss_all.item()])

    # print reconstuction loss results
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] training status, epoch: [{:04}/{:04}], loss: {:.10f}'.format(now, (epoch+1), num_epochs, reconstruction_loss_all.item()))

    # =================== save model snapshot to disk ============================

    # save trained encoder model file to disk
    encoder_model_name = "ep_{}_encoder_model.pth".format((epoch+1))
    torch.save(encoder_train.state_dict(), os.path.join("./models_test", encoder_model_name))

    # save trained decoder model file to disk
    decoder_model_name = "ep_{}_decoder_model.pth".format((epoch+1))
    torch.save(decoder_train.state_dict(), os.path.join("./models_test", decoder_model_name))

In [ ]:
# plot the training progress
plt.plot(range(0, len(losses)), losses)
plt.xlabel('[training epoch]')
plt.xlim([0, len(losses)])
plt.ylabel('[reconstruction-error]')
#plt.ylim([0.0, 1.0])
plt.title('AENN training performance')

# 10. Load fresh Data for evaluation

In [ ]:
ori_dataset = pd.read_excel(r"C:\code_repo\neuron_ai\ai_ledger\debit_data_reduced.xlsx", header=0)
ori_dataset.rename(columns=lambda x:x.replace(' ', '_'), inplace=True)

In [ ]:
#remove debit transaction that are negative numbers or not numbers
is_integer = pd.to_numeric(ori_dataset['DEBIT'], errors='coerce').notna()
ori_dataset = ori_dataset[is_integer]

ori_dataset = ori_dataset[ori_dataset['DEBIT'] >= 0]


In [ ]:
# select categorical attributes to be "one-hot" encoded
categorical_attr_names = ['Source', 'Category', 'Fund', '_Budget_Activity_Program', 'Organization', '_SGL_Account', 'Admin_Control_']

#convert any integer type categorical columns to category type to perform one hot encoding
ori_dataset['_SGL_Account'] = ori_dataset['_SGL_Account'].astype('category')

# encode categorical attributes into a binary one-hot encoded representation
ori_dataset_categ_transformed = pd.get_dummies(ori_dataset[categorical_attr_names])
ori_dataset_categ_transformed = ori_dataset_categ_transformed.astype(int)

In [ ]:
# select all the numerical attributes
numeric_attr_names = ['DEBIT']

# add a small epsilon to eliminate zero values from data for log scaling
numeric_attr = ori_dataset[numeric_attr_names] + 1e-4
numeric_attr = numeric_attr.map(apply_log)

# normalize all numeric attributes to the range [0,1]
ori_dataset_numeric_attr = (numeric_attr - numeric_attr.min()) / (numeric_attr.max() - numeric_attr.min())

In [ ]:
# merge categorical and numeric subsets
ori_subset_transformed = pd.concat([ori_dataset_categ_transformed, ori_dataset_numeric_attr], axis = 1)

In [ ]:
# convert pre-processed data to pytorch tensor
torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

# 11. Evaluate the Model

In [ ]:
# restore pretrained model checkpoint
encoder_model_name = "ep_50_encoder_model.pth"
decoder_model_name = "ep_50_decoder_model.pth"

# init training network classes / architectures
encoder_eval = encoder()
decoder_eval = decoder()

# load trained models
encoder_eval.load_state_dict(torch.load(os.path.join("models_test", encoder_model_name)))
decoder_eval.load_state_dict(torch.load(os.path.join("models_test", decoder_model_name)))

In [ ]:
# convert encoded transactional data to torch Variable
data = autograd.Variable(torch_dataset)

# set networks in evaluation mode (don't apply dropout)
encoder_eval.eval()
decoder_eval.eval()

# reconstruct encoded transactional data
reconstruction = decoder_eval(encoder_eval(data))

In [ ]:
# determine reconstruction loss - all transactions
reconstruction_loss_all = loss_function(reconstruction, data)

# print reconstruction loss - all transactions
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] collected reconstruction loss of: {:06}/{:06} transactions'.format(now, reconstruction.size()[0], reconstruction.size()[0]))
print('[LOG {}] reconstruction loss: {:.10f}'.format(now, reconstruction_loss_all.item()))

# 12. Asess on Individual Transactions

In [ ]:
# init binary cross entropy errors
reconstruction_loss_transaction = np.zeros(reconstruction.size()[0])

# iterate over all detailed reconstructions
for i in range(0, reconstruction.size()[0]):

    # determine reconstruction loss - individual transactions
    reconstruction_loss_transaction[i] = loss_function(reconstruction[i], data[i]).item()

    if(i % 10000 == 0):

        ### print conversion summary
        now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
        print('[LOG {}] collected individual reconstruction loss of: {:06}/{:06} transactions'.format(now, i, reconstruction.size()[0]))


In [ ]:
label = pd.Series(['regular'] * ori_subset_transformed.shape[0])

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# assign unique id to transactions
plot_data = np.column_stack((np.arange(len(reconstruction_loss_transaction)), reconstruction_loss_transaction))

# obtain regular transactions as well as global and local anomalies
regular_data = plot_data[label == 'regular']
global_outliers = plot_data[label == 'global']
local_outliers = plot_data[label == 'local']

# plot reconstruction error scatter plot
ax.scatter(regular_data[:, 0], regular_data[:, 1], c='C0', alpha=0.4, marker="o", label='regular') # plot regular transactions
ax.scatter(global_outliers[:, 0], global_outliers[:, 1], c='C1', marker="^", label='global') # plot global outliers
ax.scatter(local_outliers[:, 0], local_outliers[:, 1], c='C2', marker="^", label='local') # plot local outliers

# add plot legend of transaction classes
ax.legend(loc='best')

In [ ]:
# append labels to original dataset
ori_dataset['label'] = label

# inspect transactions exhibiting a reconstruction error >= 0.1
ori_dataset[reconstruction_loss_transaction >= 0.03]